UK trade in services: service type by partner country, non-seasonally adjusted

In [1]:
from gssutils import *

if is_interactive():
    import requests
    from cachecontrol import CacheControl
    from cachecontrol.caches.file_cache import FileCache
    from cachecontrol.heuristics import LastModified
    from pathlib import Path
    from io import BytesIO

    session = CacheControl(requests.Session(),
                       cache=FileCache('.cache'),
                       heuristic=LastModified())

inputURL = 'https://www.ons.gov.uk/file?uri=/businessindustryandtrade/internationaltrade/datasets/uktradeinservicesservicetypebypartnercountrynonseasonallyadjusted/current/servicetypebycountry.xlsx'
data = BytesIO(session.get(inputURL).content)


In [2]:
tab = pd.read_excel(data, header=None)
tab.rename(columns=tab.iloc[0], inplace=True)

In [3]:
tab = tab.iloc[1:, :]

In [4]:
tab = tab.drop(['Service type','Country'], axis = 1)

In [5]:
tab.head()

,Direction,Service type code,Country code,2016Q1,2016Q2,2016Q3,2016Q4,2017Q1,2017Q2,2017Q3,2017Q4,2018Q1,2018Q2
1,Exports,0,AT,228,254,278,275,292,309,321,340,288,314
2,Exports,0,BE,984,1044,1085,1154,1089,1147,1252,1167,1093,1164
3,Exports,0,BG,93,112,125,118,97,115,136,125,111,129
4,Exports,0,HR,41,48,52,55,37,52,57,55,39,45
5,Exports,0,CY,173,201,218,211,178,193,232,213,177,236


In [6]:
tab.columns.values[0] = 'Flow'
tab.columns.values[1] = 'Pink Book Services'
tab.columns.values[2] = 'ONS Partner Geography'

In [7]:
new_table = pd.melt(tab, id_vars=['Flow','Pink Book Services','ONS Partner Geography'], var_name='Period', value_name='Value')

In [8]:
new_table

,Flow,Pink Book Services,ONS Partner Geography,Period,Value
0,Exports,0,AT,2016Q1,228
1,Exports,0,BE,2016Q1,984
2,Exports,0,BG,2016Q1,93
3,Exports,0,HR,2016Q1,41
4,Exports,0,CY,2016Q1,173
5,Exports,0,CZ,2016Q1,214
6,Exports,0,DK,2016Q1,799
7,Exports,0,EE,2016Q1,13
8,Exports,0,FI,2016Q1,284
9,Exports,0,FR,2016Q1,3583


In [9]:
new_table['Period'] = 'quarter/' + new_table['Period'].astype(str).str[0:4]+ '-' +   new_table['Period'].astype(str).str[-2:]              

In [10]:
new_table['Flow'] = new_table['Flow'].str.lower()

In [11]:
new_table['Seasonal Adjustment'] =  'NSA'
new_table['Measure Type'] =  'GBP Total'
new_table['Unit'] =  'gbp-million'

In [12]:
new_table['Value'] = pd.to_numeric(new_table['Value'], errors='coerce').fillna(0)

In [13]:
new_table['Value'] = new_table['Value'].astype(int)

In [14]:
new_table = new_table[['ONS Partner Geography', 'Period','Flow','Pink Book Services', 'Seasonal Adjustment', 'Measure Type','Value','Unit' ]]

In [15]:
# if is_interactive():
#     SubstancetinationFolder = Path('out')
#     SubstancetinationFolder.mkdir(exist_ok=True, parents=True)
#     new_table.to_csv(SubstancetinationFolder / ('tidy2.csv'), index = False)

In [16]:
new_table.head()

,ONS Partner Geography,Period,Flow,Pink Book Services,Seasonal Adjustment,Measure Type,Value,Unit
0,AT,quarter/2016-Q1,exports,0,NSA,GBP Total,228,gbp-million
1,BE,quarter/2016-Q1,exports,0,NSA,GBP Total,984,gbp-million
2,BG,quarter/2016-Q1,exports,0,NSA,GBP Total,93,gbp-million
3,HR,quarter/2016-Q1,exports,0,NSA,GBP Total,41,gbp-million
4,CY,quarter/2016-Q1,exports,0,NSA,GBP Total,173,gbp-million
